In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

In [2]:
resnet34 = models.resnet34(pretrained=True)

# resnet34.train()
resnet34.eval()     # Model will be downloaded

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 97.3MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [3]:
# As ResNet accepts 224X224 input, first image is resized to this shape.
# Convert numpy/ other format to tensor
# As while training ResNet normalization is done, same noramalization is applied for image processing.
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt # txt file of imagenet classes is downloaded

--2025-01-28 05:54:03--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt.1’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0.001s  

2025-01-28 05:54:04 (6.71 MB/s) - ‘imagenet_classes.txt.1’ saved [10472/10472]



In [6]:
image_path = "/content/car.jpg"
image = Image.open(image_path).convert("RGB")  # Convert image in RGB foramt and apply transformation

input_tensor_image = transform(image).unsqueeze(0)   # unsqueeze adds new dimention in tensor as batch size = 1

In [12]:
# Function to predict class
with torch.no_grad():                        # Disable gradient calculations during inference
  outputs = resnet34(input_tensor_image)      # Outputs is list of probability of each class for each image upladed in batch
  print(len(outputs[0]))                      #
  _, predicted_class = outputs.max(1)           # Show highest probability class index
  print(f"Predicted class index : {predicted_class.item()}")

1000
Predicted class index : 817


In [13]:
with open("/content/imagenet_classes.txt") as f:            # Get class name from class index
  labels = [line.strip() for line in f.readlines()]

print(f"Predicted class index : {labels[predicted_class.item()]}")

Predicted class index : sports car
